#### Dimension table - Data Cleaning


##### Purpose of the noode book:
1. Clean duplicate, null values from dimension tables
2. Write to gold layer data storage

In [1]:
# Set the folder paths so that it can be used later
silver_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold'

StatementMeta(rewardprogram, 23, 2, Finished, Available, Finished)

In [2]:
%%sql
CREATE DATABASE IF NOT EXISTS gold_reward_program_spark
LOCATION 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold';

StatementMeta(rewardprogram, 23, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
customer_profile = spark.read.parquet(f"{silver_folder_path}/customer_profile")

StatementMeta(rewardprogram, 23, 4, Finished, Available, Finished)

In [4]:
customer_profile.show()

StatementMeta(rewardprogram, 23, 5, Finished, Available, Finished)

+------+---+--------------------+----------------+------+
|gender|age|                  id|became_member_on|income|
+------+---+--------------------+----------------+------+
|     M| 68|e2127556f4f64592b...|      2018-04-26| 70000|
|     M| 64|4b0da7e80e5945209...|      2017-09-09|100000|
|     M| 56|8dabe2142cdf4013a...|      2014-05-11| 68000|
|     F| 39|f247806381704ea58...|      2014-07-23| 56000|
|     M| 76|39dbcf43e24d41f4b...|      2014-08-31| 64000|
|     F| 73|a2b7a0b3487a4d779...|      2018-04-19| 91000|
|     M| 45|b19c8e7ac2ff40ae9...|      2017-08-15| 54000|
|     M| 53|13033baa9d294c3c9...|      2017-09-27| 96000|
|     M| 79|81e9ad5fa55a4884b...|      2016-06-27| 67000|
|     M| 65|464d2302b99b46ad8...|      2018-03-15| 61000|
|     M| 34|4d15a2a5ac134c589...|      2017-12-06| 41000|
|     F| 74|c84fe4915ef74c7f9...|      2017-12-20| 83000|
|     M| 24|d250372c924747d8a...|      2017-02-12| 61000|
|  null|118|e94a9c2e750645d0a...|      2018-07-08|  null|
|     F| 40|f9

In [5]:
from pyspark.sql.functions import col,isnan, when, count

StatementMeta(rewardprogram, 23, 6, Finished, Available, Finished)

In [6]:
customer_profile.filter(customer_profile.gender.isNull()).show()

StatementMeta(rewardprogram, 23, 7, Finished, Available, Finished)

+------+---+--------------------+----------------+------+
|gender|age|                  id|became_member_on|income|
+------+---+--------------------+----------------+------+
|  null|118|e94a9c2e750645d0a...|      2018-07-08|  null|
|  null|118|459b2f93d34d46c9a...|      2016-06-23|  null|
|  null|118|9de54389816342449...|      2018-04-22|  null|
|  null|118|7ca349e55ff544c7a...|      2017-08-28|  null|
|  null|118|cc1891e307dd48f49...|      2017-08-06|  null|
|  null|118|0d799e1906ff4387b...|      2018-04-17|  null|
|  null|118|c039e4d31d48483bb...|      2017-12-08|  null|
|  null|118|3909e7107fb7499ba...|      2017-12-21|  null|
|  null|118|3f8b8ce1768d42758...|      2015-08-12|  null|
|  null|118|c8ecfc8685bd4cbc8...|      2017-02-25|  null|
|  null|118|0bb1b63ae72141cea...|      2016-05-13|  null|
|  null|118|6cd7e548e2a04429b...|      2018-01-13|  null|
|  null|118|5aa2cd5e17c540ada...|      2017-07-01|  null|
|  null|118|70712affff0d4f389...|      2016-03-30|  null|
|  null|118|39

In [7]:
customer_profile = customer_profile.na.fill("Unknown", subset=['gender'])
customer_profile = customer_profile.na.fill(0, subset=['income'])

StatementMeta(rewardprogram, 23, 8, Finished, Available, Finished)

In [8]:
customer_profile.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ['gender', 'age', 'income']]
   ).show()

StatementMeta(rewardprogram, 23, 9, Finished, Available, Finished)

+------+---+------+
|gender|age|income|
+------+---+------+
|     0|  0|     0|
+------+---+------+



In [9]:
%%sql
Drop table if exists gold_reward_program_spark.customer_profile

StatementMeta(rewardprogram, 23, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
customer_profile.write.mode("overwrite").format("parquet").saveAsTable("gold_reward_program_spark.customer_profile")

StatementMeta(rewardprogram, 23, 11, Finished, Available, Finished)

In [11]:
offer_type = spark.read.parquet(f"{silver_folder_path}/offer_type")

StatementMeta(rewardprogram, 23, 12, Finished, Available, Finished)

In [12]:
offer_type.show()

StatementMeta(rewardprogram, 23, 13, Finished, Available, Finished)

+------+---------+---------+---------+---------+----------+--------+-------------+--------------------+
|reward|channel_1|channel_2|channel_3|channel_4|difficulty|duration|   offer_type|                  id|
+------+---------+---------+---------+---------+----------+--------+-------------+--------------------+
|    10|    email|   mobile|   social|     null|        10|    7.00|         bogo|ae264e3637204a6fb...|
|    10|      web|    email|   mobile|   social|        10|    5.00|         bogo|4d5c57ea9a6940dd8...|
|     0|      web|    email|   mobile|     null|         0|    4.00|informational|3f207df678b143eea...|
|     5|      web|    email|   mobile|     null|         5|    7.00|         bogo|9b98b8c7a33c4b65b...|
|     5|      web|    email|     null|     null|        20|   10.00|     discount|0b1e1539f2cc45b7b...|
|     3|      web|    email|   mobile|   social|         7|    7.00|     discount|2298d6c36e964ae4a...|
|     2|      web|    email|   mobile|   social|        10|   10

In [13]:
from pyspark.sql import functions as F

# List of unique channels across all columns
channels = ['web', 'email', 'mobile', 'social']

# Iterate over each channel and create a dummy column for it
for channel in channels:
    # Check if the channel appears in any of the four channel columns
    offer_type = offer_type.withColumn(f'channel_{channel}', 
                       F.when(
                           (F.col('channel_1') == channel) |
                           (F.col('channel_2') == channel) |
                           (F.col('channel_3') == channel) |
                           (F.col('channel_4') == channel), 1
                       ).otherwise(0))

# Drop original channel columns if no longer needed
offer_type = offer_type.drop('channel_1', 'channel_2', 'channel_3', 'channel_4')

# Show the resulting DataFrame
offer_type.show()

StatementMeta(rewardprogram, 23, 14, Finished, Available, Finished)

+------+----------+--------+-------------+--------------------+-----------+-------------+--------------+--------------+
|reward|difficulty|duration|   offer_type|                  id|channel_web|channel_email|channel_mobile|channel_social|
+------+----------+--------+-------------+--------------------+-----------+-------------+--------------+--------------+
|    10|        10|    7.00|         bogo|ae264e3637204a6fb...|          0|            1|             1|             1|
|    10|        10|    5.00|         bogo|4d5c57ea9a6940dd8...|          1|            1|             1|             1|
|     0|         0|    4.00|informational|3f207df678b143eea...|          1|            1|             1|             0|
|     5|         5|    7.00|         bogo|9b98b8c7a33c4b65b...|          1|            1|             1|             0|
|     5|        20|   10.00|     discount|0b1e1539f2cc45b7b...|          1|            1|             0|             0|
|     3|         7|    7.00|     discoun

In [14]:
offer_type.write.mode("overwrite").format("parquet").saveAsTable("gold_reward_program_spark.offer_type")

StatementMeta(rewardprogram, 23, 15, Finished, Available, Finished)